In [ ]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds
import random
import pathlib
import os
import cv2


#tf.config.list_physical_devices('GPU')

In [ ]:
#load data
"""
#(1)官方下載

#將MNIST 手寫數字資料讀進來,下載的資料分成training data以及test data。
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data() 
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train)) #training data 總共有60000張圖片
print(x_train[0].shape) #每張圖片（拿第一張當樣本）大小為 28x28


#分成10類(依照class來分)
num_classes = 10
class_data = []
for i in range(num_classes):
    class_data.append(x_train[y_train == i]) #NumPy Boolean Indexing
print(len(class_data))
print(class_data[0].shape) #class_data[0]:class 0的所有data
print(len(class_data[0]))
print(class_data[0][0].shape) #class 0中的第一張圖片



#平均分給n個用戶
n = 5
#mapping會是一樣的
train_data_user = np.array_split(x_train, n)
train_label_user = np.array_split(y_train, n)

print(len(train_data_user))
print(train_data_user[0].shape)
"""


#(2)自己的(所有数据存放在同一目录下，且不同类别的图片分门别类地存放在不同的子目录下)
"""
格式:
flower_photos
|
|---daisy
|
|---sunflowers
|
|---tuplips
"""

class_mapping = {'dog': 0, 'cat': 1}

train_images = []
train_labels = []

for class_name, class_label in class_mapping.items():
    class_path = 'C:/Users/sleep/Desktop/animals/'+ class_name
    for file in os.listdir(class_path):
        img = cv2.imread(os.path.join(class_path, file))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (150, 150))
        train_images.append(img)
        train_labels.append(class_label)
        
# 將train_images和train_labels合併為一個列表，並對該列表進行洗牌
combined = list(zip(train_images, train_labels))
random.shuffle(combined) #shuffle() 方法将序列的所有元素随机排序。
# 再次將洗牌後的列表分離為train_images和train_labels兩個列表
#zip(*combined)中的*符號就是用於將combined列表中的元素展開為位置引數傳遞給zip函數。
train_images, train_labels = zip(*combined)
print(len(train_images))



In [ ]:
#建model

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)), #把多维的输入一维化
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

#配置訓練方式
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
#印出神經網路架構
model.summary()
#train model
model.fit(x_train, y_train, epochs=5,batch_size=32)
#evaluate model generalization power by testing data
model.evaluate(x_test, y_test, verbose=2)
